In [36]:
import requests
from bs4 import BeautifulSoup
import ast
import pandas as pd

In [37]:
website_fxp = 'https://www.schmalz.com/en/vacuum-technology-for-automation/vacuum-components/area-gripping-systems-and-end-effectors/vacuum-area-gripping-system-fxp-fmp/area-gripping-systems-fxp?limit=50'
website_fmp = 'https://www.schmalz.com/en/vacuum-technology-for-automation/vacuum-components/area-gripping-systems-and-end-effectors/vacuum-area-gripping-system-fxp-fmp/area-gripping-systems-fmp?limit=50'

Check the headers for your session, otherwise the price will not appear unless you are logged in on the Schmalz Website. You have to paste the cookie key of the HTTP request so that the below script will work.

In [29]:
headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9',
'Cache-Control': 'no-cache',
'Connection': 'keep-alive',
'Cookie': 'gtm_internal=false; gtm_internal=false; _ga=GA1.2.556035030.1542078213; GTM_CURID=187722; PHPSESSID=bp409o9suus77d4o8o75b9eg84; _cid=tEh9E3yJEMp5rcOo; countryCluster=US; frontend_cid=IZJ9Ys5yxO3aT3kX; _gid=GA1.2.1671424653.1543214833; currentStoreId=2; frontend=clo6et0glairm8igc101vmum14; schmalz_current_currency=USD; schmalz_current_currency_symbol=%24; reference_pl_id=WEB-US-USD; GTM_LOGIN=1; _gat_UA-42003922-1=1',
'Host': 'www.schmalz.com',
'Pragma': 'no-cache',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}

In [30]:
data_fxp = requests.get(website_fxp,headers=headers).text
data_fmp = requests.get(website_fmp,headers=headers).text
soup_fxp = BeautifulSoup(data_fxp,'lxml')
soup_fmp = BeautifulSoup(data_fmp,'lxml')
#Total number of items
tot_items_fxp = int(soup_fxp.find('input',id='jsCollSize')['value'])
tot_items_fmp = int(soup_fmp.find('input',id='jsCollSize')['value'])
number_pages_fxp = tot_items_fxp//50+1
number_pages_fmp = tot_items_fmp//50+1

In [33]:
df_list = []
for page_number in (range(1,number_pages_fxp+1)):
    website_fxp_page = website_fxp+f'&p={page_number}'
    data = requests.get(website_fxp_page,headers=headers).text
    soup = BeautifulSoup(data,'lxml')
    
    #information-block soup variable
    info_block = soup.find_all('div',attrs={'class':'information-block'})
    
    #create sub_dataframes with pricing information 
    names = [tag.find('a',attrs={'class':'h3'})['title'] for tag in info_block]
    df_price_list = [tag.find('span',attrs={'class':'jsSchmalzPrice'}).text.strip() for tag in info_block]
    df_price_list = [pd.DataFrame(ast.literal_eval(s),columns=['reference_pl_id','value']) for s in df_price_list]
    
    #replace commas with points in all dataframes and append the names of the product
    for name,price_df in zip(names,df_price_list):
        price_df['value'] = price_df['value'].apply(lambda x: x.replace(',','.'))
        price_df['name'] = name
        df_list.append(price_df)

for page_number in (range(1,number_pages_fmp+1)):
    website_fmp_page = website_fmp+f'&p={page_number}'
    data = requests.get(website_fmp_page,headers=headers).text
    soup = BeautifulSoup(data,'lxml')
    
    #information-block soup variable
    info_block = soup.find_all('div',attrs={'class':'information-block'})
    #create sub_dataframes with pricing information 
    names = [tag.find('a',attrs={'class':'h3'})['title'] for tag in info_block]
    df_price_list = [tag.find('span',attrs={'class':'jsSchmalzPrice'}).text.strip() for tag in info_block]
    df_price_list = [pd.DataFrame(ast.literal_eval(s),columns=['reference_pl_id','value']) for s in df_price_list]
    
    #replace commas with points in all dataframes and append the names of the product
    for name,price_df in zip(names,df_price_list):
        price_df['value'] = price_df['value'].apply(lambda x: x.replace(',','.'))
        price_df['name'] = name
        df_list.append(price_df)

In [214]:
#concatenate dataframe, add currency table

In [34]:
schmalz_pricing = pd.concat(df_list).reset_index(drop=True)
schmalz_pricing['currency'] = schmalz_pricing['reference_pl_id'].apply(lambda x:x[-3:])
schmalz_pricing = schmalz_pricing[['name','value','currency','reference_pl_id']]

In [35]:
schmalz_pricing.to_csv(r'C:\Users\Stephane\Desktop\sc_price.csv',index=False)